In [1]:
import torch

In [2]:
from torch import nn
from torch.nn import functional as f

In [3]:
from torch.nn import Embedding,Linear,Dropout,LayerNorm,MultiheadAttention,Module,ReLU

In [4]:
from torch.nn.functional import relu,softmax

In [5]:
class FeedForward(Module):
    def __init__(self,d_model,units=2048):
        super(FeedForward,self).__init__()
        self.linear1=Linear(d_model,units)
        self.relu=ReLU()
        self.dropout=Dropout(0.1)
        self.linear2=Linear(units,d_model)
        
    def forward(self,x):
        linear1_x=self.linear1(x)
        relu_x=self.relu(linear1_x)
        dropout_x=self.dropout(relu_x)
        linear2_x=self.linear2(dropout_x)
        return linear2_x

In [6]:
class EncoderLayer(Module):
    def __init__(self,num_heads,d_model,src_len):
        super(EncoderLayer,self).__init__()
        self.mha=MultiheadAttention(d_model,num_heads)
        self.norm1=LayerNorm((src_len,d_model))
        self.dropout1=Dropout(0.1)
        self.ffn=FeedForward(d_model)
        self.norm2=LayerNorm((src_len,d_model))
        self.dropout2=Dropout(0.1)
        
    def forward(self,x):
        cv,attn_wts=self.mha(x,x,x)
        cv_norm=self.norm1(cv+x)
        cv_norm1_drop1=self.dropout1(cv_norm)
        cv_norm1_drop1_ffn=self.ffn(cv_norm1_drop1)
        cv_norm1_drop1_ffn_norm2=self.norm2(cv_norm1_drop1_ffn+cv_norm1_drop1)
        output=self.dropout2(cv_norm1_drop1_ffn_norm2)
        return output

In [7]:
class Encoder(Module):
    def __init__(self,n_layers,num_heads,d_model,src_len,src_vocab_size):
        super(Encoder,self).__init__()
        self.word_embedding=Embedding(src_vocab_size,d_model)
        self.position_embedding=Embedding(src_len,d_model)
        self.encoder_layer=[EncoderLayer(num_heads,d_model,src_len) for _ in range(n_layers)]
        
    def forward(self,x):
        word_embed=self.word_embedding(x)
        pos_embed=self.position_embedding(torch.tensor([i for i in range(x.size(-1))]))
        embed=word_embed+pos_embed
        for layer in self.encoder_layer:
            embed=layer(embed)
        return embed

In [8]:
import numpy

In [9]:
x=torch.randn((10,15)).to(torch.long)

C:\Users\Rambabu\AppData\Local\Temp\ipykernel_7428\2919096325.py:1: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:77.)
  x=torch.randn((10,15)).to(torch.long)


In [158]:
x1=x.cpu().long()

In [159]:
x1.shape

torch.Size([10, 15])

In [160]:
type(x)

torch.Tensor

In [162]:
en=Encoder(12,8,512,15,50)

In [122]:
out.shape

torch.Size([10, 15, 512])

In [ ]:
out=en(x)

In [91]:
from torchsummary import summary

In [92]:
x.shape

torch.Size([10, 15])